In [1]:
import matplotlib
matplotlib.use('agg')
import pyart
from matplotlib import pyplot as plt

from netCDF4 import num2date, date2num
import numpy as np
from time import time
import os
#here is the key import! 
from IPython.parallel import Client

In [22]:
def do_grid_map_gates_to_grid(radar_fname):
    import pyart
    from matplotlib import pyplot as plt
    from netCDF4 import num2date, date2num
    import numpy as np
    from time import time
    import os
    if True:
        radar = pyart.io.read(radar_fname)
        
        rain_z = radar.fields['reflectivity']['data'].copy()
        z_lin = 10.0**(radar.fields['reflectivity']['data']/10.)
        rain_z = (z_lin/300.0)**(1./1.4)  #Z=300 R1.4
        radar.add_field_like('reflectivity', 'rain_z',  rain_z, replace_existing = True)
        radar.fields['rain_z']['units'] = 'mm/h'
        radar.fields['rain_z']['standard_name'] = 'rainfall_rate'
        radar.fields['rain_z']['long_name'] = 'rainfall_rate_from_z'
        radar.fields['rain_z']['valid_min'] = 0
        radar.fields['rain_z']['valid_max'] = 500
        grid = pyart.map.grid_from_radars(
             (radar,), grid_shape=(35, 1001, 1001),
            grid_limits=((0, 17000),(-100000, 100000), (-100000, 100000)),
            fields=radar.fields.keys(), gridding_algo="map_gates_to_grid",
            weighting_function='BARNES')
        dts = num2date(grid.axes['time']['data'], grid.axes['time']['units'])
        sstr = dts[0].strftime('%Y%m%d_%H%M%S')
        pyart.io.write_grid('/data/chicago_stat/' + 'grid_250_'+sstr+'.nc', grid)
        max_lat = 42.
        min_lat = 40. 
        min_lon = -89 
        max_lon = -87
        myd = pyart.graph.RadarMapDisplay(radar)
        fig = plt.figure(figsize = [18,10])
        myd.plot_ppi_map( 'rain_z', vmin = 0, vmax = 100, 
                         resolution = 'h', max_lat = max_lat, 
                         min_lat = min_lat, min_lon = min_lon, max_lon = max_lon)
        m = myd.basemap
        m.drawparallels(np.arange(min_lat,max_lat, 1),labels=[1,0,0,0])
        m.drawmeridians(np.arange(min_lon,max_lon, 1),labels=[0,0,0,1])
        m.drawrivers()
        m.drawcounties()
        m.drawstates()
        m.drawmapscale(-88., 41.55, -88., 41.55, 10, barstyle='fancy', fontcolor='k', fillcolor1='b', fillcolor2='k')
        myd.plot_point( -87.9706,41.6815, label_text = 'Argonne Lab', label_offset = (0.0,0.0) )
        plt.savefig('/data/chicago_stat/'+ 'radar_'+sstr+'.png')
        plt.figure(figsize = [15,15])

        display = pyart.graph.GridMapDisplay(grid)
        display.plot_basemap(lat_lines=np.arange(min_lat,max_lat,1),
                             lon_lines=np.arange(min_lon, max_lon, 1),
                             resolution='h')
        display.plot_grid('rain_z', vmin=0, vmax=100)
        xcf,ycf = display.basemap(-87.9706,41.6815)
        display.basemap.plot(xcf,ycf,'ro')
        plt.text(xcf+2000.,ycf+2000., 'Argonne Lab')
        display.basemap.drawcounties()
        display.basemap.drawrivers()

        display.basemap.drawmapscale(-88., 41.55, -88., 41.55, 10, barstyle='fancy', fontcolor='k', fillcolor1='b', fillcolor2='k')
        display.plot_colorbar()
        plt.savefig('/data/chicago_stat/'+ 'mapped_250_'+sstr+'.png')
        display = pyart.graph.GridMapDisplay(grid, debug=True)

        fig = plt.figure(figsize=[15, 8])

        # panel sizes
        map_panel_axes = [0.05, 0.05, .4, .80]
        x_cut_panel_axes = [0.55, 0.10, .4, .30]
        y_cut_panel_axes = [0.55, 0.50, .4, .30]
        colorbar_panel_axes = [0.05, 0.90, .4, .03]

        # parameters
        level = 2
        vmin = 0.10
        vmax = 180
        lat = 41.6815
        lon = -87.9706

        # panel 1, basemap, radar reflectivity and NARR overlay
        ax1 = fig.add_axes(map_panel_axes)
        display.plot_basemap(lat_lines=np.arange(min_lat,max_lat,1),
                             lon_lines=np.arange(min_lon, max_lon, 1),
                             resolution='h')
        display.plot_grid('reflectivity', level=level, vmin=-8, vmax=64)
        display.plot_crosshairs(lon=lon, lat=lat)
        cbax = fig.add_axes(colorbar_panel_axes)
        display.plot_colorbar(cax=cbax)

        # panel 2, longitude slice.
        ax2 = fig.add_axes(x_cut_panel_axes)
        display.plot_longitude_slice('reflectivity', lon=lon, lat=lat, vmin=-8, vmax=64)
        ax2.set_xlabel('Distance from Argonne(km)')

        # panel 3, latitude slice
        ax3 = fig.add_axes(y_cut_panel_axes)
        display.plot_latitude_slice('reflectivity', lon=lon, lat=lat, vmin=-8, vmax=64)

        # add a title
        slc_height = grid.axes['z_disp']['data'][level]
        dts = num2date(grid.axes['time']['data'], grid.axes['time']['units'])
        datestr = dts[0].strftime('%H:%M Z on %Y-%m-%d')
        title = 'Sliced at ' + str(slc_height) + ' meters at ' + datestr
        fig.text(0.5, 0.9, title)
        plt.savefig('/data/chicago_stat/'+ 'mapped_3d_250_'+sstr+'.png')
        plt.close(fig)

    return 0

In [23]:
state = do_grid_map_gates_to_grid('/data/KLOT20150610_124714_V06')

Maximum latitude:  42.5084447087
Maximum longitude:  -86.8633212602
Minimum latitude:  40.6937384946
Minimum longitute:  -89.3055630172
x_cut:  9436.79472952 y_cut:  8526.21483723
x_index 547
y_index 543
x_cut:  9436.79472952 y_cut:  8526.21483723
x_index 547
y_index 543


In [3]:
md = '/lcrc/group/earthscience/radar/nexrad/chicago_floods'
idir = '/media/scollis/data/chi/'
filelist = os.listdir('/media/scollis/data/chi/')
good_files = []
for fl in filelist:
    if 'KLOT' in fl:
        good_files.append(idir + fl)
good_files.sort()

OSError: [Errno 2] No such file or directory: '/media/scollis/data/chi/'

In [4]:
print(good_files[10])

/media/scollis/data/chi/KLOT20130418_004639_V06


In [7]:
My_Cluster = Client()
My_View = My_Cluster[:]
print My_View
print len(My_View)

<DirectView [0, 1, 2, 3,...]>
15


In [ ]:
#Turn off blocking so all engines can work async
My_View.block = False

#on all engines do an import of Py-ART
My_View.execute('import matplotlib')
My_View.execute('matplotlib.use("agg")')




#Map the code and input to all workers
result = My_View.map_async(do_grid_map_gates_to_grid, good_files)

#Reduce the result to get a list of output
qvps = result.get()


In [ ]:
pass

In [ ]:
def do_grid_map_gates_to_grid(radar_fname):
    import pyart
    from matplotlib import pyplot as plt
    from netCDF4 import num2date, date2num
    import numpy as np
    from time import time
    import os
    md = '/lcrc/group/earthscience/radar/nexrad/chicago_floods/'
    try:
        radar = pyart.io.read(radar_fname)
        rain_z = radar.fields['reflectivity']['data'].copy()
        z_lin = 10.0**(radar.fields['reflectivity']['data']/10.)
        rain_z = (z_lin/300.0)**(1./1.4)  #Z=300 R1.4
        radar.add_field_like('reflectivity', 'rain_z',  rain_z, replace_existing = True)
        radar.fields['rain_z']['units'] = 'mm/h'
        radar.fields['rain_z']['standard_name'] = 'rainfall_rate'
        radar.fields['rain_z']['long_name'] = 'rainfall_rate_from_z'
        radar.fields['rain_z']['valid_min'] = 0
        radar.fields['rain_z']['valid_max'] = 500
        grid = pyart.map.grid_from_radars(
             (radar,), grid_shape=(1, 501, 501),
            grid_limits=((0, 0),(-50000, 50000), (-50000, 50000)),
            fields=radar.fields.keys(), gridding_algo="map_gates_to_grid",
            weighting_function='BARNES')
        dts = num2date(grid.axes['time']['data'], grid.axes['time']['units'])
        sstr = dts[0].strftime('%Y%m%d_%H%M%S')
        pyart.io.write_grid(md + 'grid_250_'+sstr+'.nc', grid)
        myd = pyart.graph.RadarMapDisplay(radar)
        fig = plt.figure(figsize = [18,10])
        myd.plot_ppi_map( 'rain_z', vmin = 0, vmax = 100, 
                         resolution = 'h', max_lat = 41.8, 
                         min_lat = 41.25, min_lon = -88.3, max_lon = -87.5)
        m = myd.basemap
        m.drawparallels(np.linspace(41, 42, 9),labels=[1,0,0,0])
        m.drawmeridians(np.linspace(-88.4, -87, 8),labels=[0,0,0,1])
        m.drawrivers()
        m.drawcounties()
        m.drawstates()
        m.drawmapscale(-88., 41.55, -88., 41.55, 10, barstyle='fancy', fontcolor='k', fillcolor1='b', fillcolor2='k')
        myd.plot_point( -87.9706,41.6815, label_text = 'Argonne Lab', label_offset = (0.0,0.0) )
        plt.savefig(md+ 'radar_'+sstr+'.png')
        plt.figure(figsize = [15,15])
        max_lat = 43
        min_lat = 41.5
        min_lon = -88.3
        max_lon = -87.5
        display = pyart.graph.GridMapDisplay(grid)
        display.plot_basemap(lat_lines=np.arange(min_lat,max_lat,.1),
                             lon_lines=np.arange(min_lon, max_lon, .1),
                             resolution='h')
        display.plot_grid('rain_z', vmin=0, vmax=100)
        xcf,ycf = display.basemap(-87.9706,41.6815)
        display.basemap.plot(xcf,ycf,'ro')
        plt.text(xcf+2000.,ycf+2000., 'Argonne Lab')
        display.basemap.drawcounties()
        display.basemap.drawrivers()

        display.basemap.drawmapscale(-88., 41.55, -88., 41.55, 10, barstyle='fancy', fontcolor='k', fillcolor1='b', fillcolor2='k')
        display.plot_colorbar()
        plt.savefig(md+ 'mapped_250_'+sstr+'.png')
    except:
        pass
    return 0

In [ ]:
md = '/lcrc/group/earthscience/radar/nexrad/chicago_floods/'
idir = md
filelist = os.listdir(md)
good_files = []
for fl in filelist:
    if 'KLOT' in fl:
        good_files.append(idir + fl)
good_files.sort()

In [24]:
radar = pyart.io.read('/data/KLOT20150610_124714_V06')